TODOs<br>
-> Train a language model to better break / combine words<br>
-> Analyse using idf values<br>
-> Write rules for extracting verb between two objs/two subjs/obj and subj<br>
-> Use AllenNLP for constituency parsing and display the Tree using NLTK on readme <br>
(See question generation course on how to use it)
-> Use Stanford CoreNLP

In [6]:
import pandas as pd
import spacy
from spacy import displacy
nlp = spacy.load("en_core_web_sm")

import arxiv

# text=arxiv.query(id_list=['1802.04712'])[0]['summary']
text='The dominant sequence transduction models are based on complex recurrent or convolutional neural networks in an encoder-decoder configuration. The best performing models also connect the encoder and decoder through an attention mechanism. We propose a new simple network architecture, the Transformer, based solely on attention mechanisms, dispensing with recurrence and convolutions entirely. Experiments on two machine translation tasks show these models to be superior in quality while being more parallelizable and requiring significantly less time to train. Our model achieves 28.4 BLEU on the WMT 2014 English-to-German translation task, improving over the existing best results, including ensembles by over 2 BLEU. On the WMT 2014 English-to-French translation task, our model establishes a new single-model state-of-the-art BLEU score of 41.8 after training for 3.5 days on eight GPUs, a small fraction of the training costs of the best models from the literature. We show that the Transformer generalizes well to other tasks by applying it successfully to English constituency parsing both with large and limited training data.'

def clean_text(text):
    text=text.replace('\n', ' ')
    
    #TODO: Currently doesn't handle pre-trained
    text=text.replace('-', ' ')
    return text

text=clean_text(text)
doc = nlp(text)
# displacy.serve(doc, style="dep")
# displacy.serve(doc, style="ent")

ImportError: cannot import name 'get_array_module' from 'thinc.api' (/home/hs/Desktop/Projects/projects_env/lib/python3.8/site-packages/thinc/api.py)

In [7]:
tok_tags=[(x, x.dep_, x.pos_) for x in doc]
tok_tags

NameError: name 'doc' is not defined

In [ ]:
def get_raw_entities(tok_tags):
    entities, spans = [], []
    compound_toks, compound_spans = [], []
    for idx, (tok, tag, pos) in enumerate(tok_tags):
        if 'subj' in tag or 'obj' in tag or pos=='PROPN':
            # not a multi-word subject
            if not compound_toks:
                # ignore if its a single-word pronoun
                if pos!='PRON':
                    entities.append([str(tok)])
                    spans.append([idx])
            else:
                entities.append(compound_toks+[str(tok)])
                spans.append(compound_spans+[idx])
                compound_toks, compound_spans=[], []
        elif tag == 'compound' or 'mod' in tag:
            compound_toks.append(str(tok))
            compound_spans.append(idx)
        else:
            compound_toks, compound_spans=[], []

    assert len(compound_toks)==len(compound_spans)
    for idx, span in enumerate(spans):
        assert [str(x[0]) for x in tok_tags[span[0]:span[-1]+1]] == entities[idx]
    
    return entities, spans

In [ ]:
raw_entities, raw_spans = get_raw_entities(tok_tags)
raw_entities

In [ ]:
from nltk.corpus import words
english_words = set(words.words())

from nltk.stem import WordNetLemmatizer 
lemmatize = WordNetLemmatizer().lemmatize

from nltk.stem import PorterStemmer
stem = PorterStemmer().stem

def clean_raw_entities(raw_entities, raw_spans):
    final_entities, final_spans=[], []
    for idx, ent in enumerate(raw_entities):
        if len(ent)==1 and (lemmatize(ent[0]).lower() in english_words or stem(ent[0].lower()) in english_words):
            continue
        
        # appending as a string
        final_entities.append(' '.join(ent))
        final_spans.append(raw_spans[idx])

    return final_entities, final_spans

In [ ]:
cleaned_entities, cleaned_spans=clean_raw_entities(raw_entities, raw_spans)
cleaned_entities

In [ ]:
%load_ext autoreload
%autoreload 2
from utils import render

In [ ]:
render(cleaned_spans, tok_tags, colour='#AEDEAC', debug=False)

In [8]:
text='The X are based on complex recurrent or convolutional Y in an encoder decoder configuration.'

In [23]:
from openie import StanfordOpenIE

annotated=None
with StanfordOpenIE() as client:
    print('Text: %s.' % text)
    annotated=client.annotate(text, properties={"annotators":"openie",
                                "splitter.disable":"true"})
    for triple in client.annotate(text):
        print('|-', triple)

Text: The X are based on complex recurrent or convolutional Y in an encoder decoder configuration..
Starting server with command: java -Xmx8G -cp /home/hs/stanfordnlp_resources/stanford-corenlp-full-2018-10-05/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 60000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-f43a181bf1254ad8.props -preload openie
|- {'subject': 'X', 'relation': 'are', 'object': 'based on recurrent in encoder decoder configuration'}
|- {'subject': 'X', 'relation': 'are', 'object': 'based'}
|- {'subject': 'X', 'relation': 'are based on', 'object': 'recurrent'}
|- {'subject': 'X', 'relation': 'are', 'object': 'based on complex recurrent in encoder decoder configuration'}
|- {'subject': 'X', 'relation': 'are based in', 'object': 'encoder decoder configuration'}
|- {'subject': 'X', 'relation': 'are based on', 'object': 'complex recurrent'}


In [24]:
pairs=pd.DataFrame(annotated)

pairs

,subject,relation,object
0,X,are,based on recurrent in encoder decoder configur...
1,X,are,based
2,X,are based on,recurrent
3,X,are,based on complex recurrent in encoder decoder ...
4,X,are based in,encoder decoder configuration
5,X,are based on,complex recurrent


In [ ]:
import networkx as nx
import matplotlib.pyplot as plt


def draw_kg(pairs):
    k_graph = nx.from_pandas_edgelist(pairs, 'subject', 'object',
            create_using=nx.MultiDiGraph())
    node_deg = nx.degree(k_graph)
    layout = nx.spring_layout(k_graph, k=0.15, iterations=20)
    plt.figure(num=None, figsize=(10, 10), dpi=80)
    nx.draw_networkx(
        k_graph,
        node_size=[int(deg[1]) * 500 for deg in node_deg],
        arrowsize=20,
        linewidths=1.5,
        pos=layout,
        edge_color='red',
        edgecolors='black',
        node_color='white',
        )
    labels = dict(zip(list(zip(pairs.subject, pairs.object)),
                  pairs['relation'].tolist()))
    nx.draw_networkx_edge_labels(k_graph, pos=layout, edge_labels=labels,
                                 font_color='red')
    plt.axis('off')
    plt.show()

draw_kg(pairs)

In [ ]:
from IPython.core.display import display, HTML

HTML('''<a href="http://example.com">
  <div>
     anything
  </div>
</a>''')